<a href="https://colab.research.google.com/github/ask-1710/FIRE2021-OffensiveLanguageDetection/blob/main/Malayalam_OffensiveLanguageDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
tokenizer_train  = Tokenizer(num_words = 100, oov_token="<OOV>")
data_train = pd.read_excel('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/Malayalam_offensive_data_Training-YT.xlsx', names=["Name","Text","result"], index_col=False)
corpus_train=[]
data_train['Text'] = data_train['Text'].astype('str')
data_train['Text'] = data_train['Text'].apply(lambda x: x.lower())
for i in data_train['Text'].tolist():
  if(str(i)!='nan'):
    corpus_train.append(i)

label_train=[]
for i in data_train['result'].tolist():
  if(str(i)!='nan'):
    label_train.append(i)


tokenizer_train .fit_on_texts(corpus_train)
total_words_train  = len(tokenizer_train .word_index) + 1

print('Mapping of Words and numbers: ')
print(tokenizer_train .word_index)
print('Number of words for training: ')
print(total_words_train )


tokenizer = tokenizer_train
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/malayalam_hasoc_tanglish_test_withlabels.tsv',sep='\t',header=0, names=["Name","Text","result"], index_col=False)
corpus=[]
data['Text'] = data['Text'].astype('str')
data['Text'] = data['Text'].apply(lambda x:x.lower())
for i in data['Text'].tolist():
  if(str(i)!='nan'):
    corpus.append(i)

label=[]
for i in data['result'].tolist():
  if(str(i)!='nan'):
    label.append(i)


Mapping of Words and numbers: 
{'<OOV>': 1, 'oru': 2, 'user': 3, 'url': 4, 'ee': 5, 'ninte': 6, 'nee': 7, 'ennu': 8, 'aanu': 9, 'thanne': 10, 'aa': 11, 'onnum': 12, 'anu': 13, 'njan': 14, 'okke': 15, 'alla': 16, 'nalla': 17, 'e': 18, 'illa': 19, 'athu': 20, 'ulla': 21, 'poyi': 22, 'ath': 23, 'pole': 24, 'onnu': 25, 'ith': 26, 'vere': 27, 'alle': 28, 'enna': 29, 'ne': 30, 'avar': 31, 'ithu': 32, 'poda': 33, 'und': 34, 'bro': 35, 'ente': 36, 'ellam': 37, 'video': 38, 'ingane': 39, 'ella': 40, 'mone': 41, 'parayan': 42, 'ningal': 43, 'but': 44, 'mole': 45, 'kure': 46, 'a': 47, 'paranju': 48, 'sir': 49, 'kond': 50, 'karyam': 51, 'enthu': 52, 'aan': 53, 'ano': 54, 'entha': 55, 'ni': 56, 'eee': 57, 'support': 58, 'muslim': 59, 'para': 60, 'podi': 61, 'ippo': 62, 'appo': 63, 'myre': 64, 'il': 65, 'cinema': 66, 'pani': 67, 'polum': 68, 'um': 69, 'enth': 70, 'enn': 71, 'koode': 72, 'mathram': 73, 'aano': 74, 'vendi': 75, 'ninne': 76, 'sathyam': 77, 'engane': 78, 'kondu': 79, 'swantham': 80, 'co

In [4]:
input_sequences = []
labels = []
for line in range(len(corpus)):
  if label[line] == 'OFF':
    l = 0
  else:
    l = 1
  token_list = tokenizer.texts_to_sequences([corpus[line]])[0]
  input_sequences.append(token_list)
  labels.append(l)
  
# pad sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=100, padding='pre'))

# create predictors and label
xs = input_sequences[:,:]
ys = tf.keras.utils.to_categorical(labels, num_classes=2)
print('Number of validation samples:', end=' ')
print(len(ys),len(xs))

#TRAIN
labels = []
input_sequences = []
for line in range(len(corpus_train)):
  if label_train[line] == 'OFF':
    l = 0
  else:
    l = 1

  token_list_train = tokenizer.texts_to_sequences([corpus_train[line]])[0]
  input_sequences.append(token_list_train)
  labels.append(l)
  
# pad sequences 
input_sequences = np.array(pad_sequences(input_sequences, maxlen=100, padding='pre'))
xs_train= input_sequences[:,:]
ys_train = tf.keras.utils.to_categorical(labels, num_classes=2)
print('Number of training samples : ', len(ys_train),len(xs_train))

Number of validation samples: 950 950
Number of training samples :  3999 3999


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint 

In [6]:
model = Sequential()
model.add(Embedding(total_words_train, 100, input_length=100))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(2, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [7]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

In [8]:
ModelCheckpointcheckpoint = ModelCheckpoint('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model_mal.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

In [9]:
history = model.fit(xs_train, ys_train, epochs=10, verbose=1,validation_data=(xs, ys), callbacks=[earlystop, ModelCheckpointcheckpoint])

Epoch 1/10
125/125 [==============================] - 12s 59ms/step - loss: 0.6960 - accuracy: 0.4911 - val_loss: 0.6900 - val_accuracy: 0.5147

Epoch 00001: val_loss improved from inf to 0.69003, saving model to /content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model_mal.h5
Epoch 2/10
125/125 [==============================] - 6s 47ms/step - loss: 0.6893 - accuracy: 0.5199 - val_loss: 0.6781 - val_accuracy: 0.5484

Epoch 00002: val_loss improved from 0.69003 to 0.67812, saving model to /content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model_mal.h5
Epoch 3/10
125/125 [==============================] - 6s 47ms/step - loss: 0.6823 - accuracy: 0.5551 - val_loss: 0.6674 - val_accuracy: 0.5495

Epoch 00003: val_loss improved from 0.67812 to 0.66745, saving model to /content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model_mal.h5
Epoch 4/10
125/125 [==============================] - 6s 47ms/step - loss: 0.6687 - a

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1926100   
_________________________________________________________________
bidirectional (Bidirectional (None, 300)               301200    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 2,227,902
Trainable params: 2,227,902
Non-trainable params: 0
_________________________________________________________________


In [11]:
import re

In [12]:
categories = ['Off','Not']

In [13]:
test_df = pd.read_csv('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/test/mal-offensive-withoutlabels - Sheet1.csv', names=["id","text","label"], index_col=False)

In [14]:
test_df['text'] = test_df['text'].apply(lambda x: x.lower())

Demotify

In [15]:
# !pip install emoji
# import emoji
#!/usr/bin/env python
# def deEmojify(text):
#     regrex_pattern = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#     return regrex_pattern.sub(r'',text)
# test_df['text'] = test_df['text'].apply(lambda x:deEmojify(x))

In [16]:
# !pip install indic-nlp-library
# from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator
# from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

In [17]:
!pip install langdetect

     |████████████████████████████████| 981 kB 3.8 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=860a11861d48dad347e1db490bbd8b0704615e395b3e481df905a6ad8578c0f7
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [18]:
from langdetect import detect

In [19]:
!pip install indic-transliteration

     |████████████████████████████████| 119 kB 5.4 MB/s 
     |████████████████████████████████| 662 kB 22.8 MB/s 
     |████████████████████████████████| 904 kB 36.4 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20


In [20]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

In [21]:
def testing(test_sentence):
  try:
    if test_sentence and detect(test_sentence) == "ml":
      test_sentence = transliterate(test_sentence, sanscript.MALAYALAM, sanscript.ITRANS)
      test_sentence = test_sentence.lower()
      print('MAL', test_sentence)
      
    tokens = tokenizer.texts_to_sequences(test_sentence)[0]
    if len(tokens)>0:
      ans = model.predict(tokens)
      return categories[np.argmax(ans[0])]
    else:
      print('empty tokens')
      return 'Not'
  except Exception as e:
    print(e)
    print(test_sentence)
    return 'Not'

In [22]:
test_df['label'] = test_df['text'].apply(lambda x: testing(x))

In [23]:
len(test_df[test_df['label']=='Not']) / len(test_df)

0.963

In [24]:
len(test_df[test_df['label']=='Off']) / len(test_df)

0.037

In [25]:
test_df.to_csv('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/test/mal-results.csv', index=False)